In [1]:
from google.colab import files, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.3MB/s 
     |████████████████████████████████| 890kB 19.4MB/s 
     |████████████████████████████████| 3.2MB 22.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=697e267bdde6734889e4620c3509cfa480a86c3f253a2bea9378706f931472e9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [4]:
import pickle
import os
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [5]:
data_text_features = load_obj("/content/drive/MyDrive/DDI/test_features/data_text_features.pkl")
id_list = load_obj("/content/drive/MyDrive/DDI/test_features/id_list.pkl")
id_list = [int(_) for _ in id_list]
test_id = load_obj("/content/drive/MyDrive/DDI/test_features/test_id_w_ground_truth.pkl")

In [6]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def mean_pooling_2(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, [0,1])
    sum_mask = torch.clamp(input_mask_expanded.sum([0,1]), min=1e-9)
    return sum_embeddings.unsqueeze(0) / sum_mask.unsqueeze(0)

In [7]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")

In [9]:
for sl in range(0,len(test_id),100):
  bert_embedding=[]
  bert_embedding_2 = []
  for i in range(sl,min(sl+100,len(test_id)):
      data_list = []
      data_list_2 = []
      data = data_text_features[id_list.index(test_id[i])]
      print(i)
      
      for j in range(2,len(data)):
          if j == 4:

              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              
              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j ==5:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)

              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j == 6:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 1000:
                  strip_index = 0
                  strip_count = 0
                  for si in range(len(striped_list)):
                      strip_count += len(striped_list[si].split())
                      if strip_count < 1000:
                          strip_index = si 
                      else:
                          break
                  encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
              else:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1].mean(0,keepdim=True))
              continue
          else:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1])
              continue
      bert_embedding.append(np.array(torch.cat(data_list,0)))
      bert_embedding_2.append(np.array(torch.cat(data_list_2,0)))
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/bert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/bert_2/"+str(savedex),bert_np_2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: ignored

In [10]:
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/bert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/bert_2/"+str(savedex),bert_np_2)